<h1 style="color:#3df22F"> Skip Lists </h1>

<h3 style="color:#3df22F"> Recap: </h3>
<div style="margin-top: -10px;">

Skip List is a hierarchical structure efficient search and insertion operations. (O(log n) average complexity). <br>
Based on multiple linked lists where each element is linked to a cell of a previous list, but not necessarily the adjacent previous list. <br>
For each list, each successive subsequence skipping over fewer elements than the previous one. <br>
</div>

In [3]:
import random

In [4]:
class SkipNode:
    def __init__(self, key, value, level):
        self.key = key           
        self.value = value       
        # List to store references to next nodes at different levels
        self.forward = [None] * (level + 1)  

class SkipList:
    def __init__(self, max_level=16, p=0.5):
        self.max_level = max_level  
        # Set the probability for generating random levels
        self.p = p                 
        # Initialize head node with dummy values
        self.head = self._create_node(None, None, max_level) 
        # Set the current maximum level in the skip list
        self.level = 0            

    def _create_node(self, key, value, level):
        """ Create a new SkipNode."""
        return SkipNode(key, value, level)

    def _random_level(self):
        """ Generate a random level for a new node. """
        level = 0
        while random.random() < self.p and level < self.max_level:
            level += 1
        return level

    def insert(self, key, value):
        """ Insert a key-value pair into the skip list. """
        update = [None] * (self.max_level + 1)
        current = self.head

        #### Traverse the skip list to find the appropriate location for insertion
        for level in range(self.level, -1, -1):
            while current.forward[level] and current.forward[level].key < key:
                current = current.forward[level]
            update[level] = current
        
        ##### Generate a random level for the new node
        level = self._random_level()  
        if level > self.level:
            for i in range(self.level + 1, level + 1):
                update[i] = self.head
            self.level = level

        new_node = self._create_node(key, value, level) 
        for i in range(level + 1):
            new_node.forward[i] = update[i].forward[i]
            # Update forward references
            update[i].forward[i] = new_node  

    def search(self, key):
        """  Search for a value associated with a key in the skip list. """
        current = self.head
        for level in range(self.level, -1, -1):
            while current.forward[level] and current.forward[level].key < key:
                current = current.forward[level]

        if current.forward[0] and current.forward[0].key == key:
            # Key found, return the associated value
            return current.forward[0].value  
        else:
            # Key not found
            return None  

    def delete(self, key):
        """ Delete a key-value pair from the skip list """
        update = [None] * (self.max_level + 1)  
        current = self.head

        #### Traverse the skip list to find the node to be deleted
        for level in range(self.level, -1, -1):
            while current.forward[level] and current.forward[level].key < key:
                current = current.forward[level]
            update[level] = current

        if current.forward[0] and current.forward[0].key == key:
            deleted_node = current.forward[0]
            ##### Update forward references to remove the node
            for level in range(self.level + 1):
                if update[level].forward[level] != deleted_node:
                    break
                update[level].forward[level] = deleted_node.forward[level]

            ## Decrease the maximum level if needed
            while self.level > 0 and self.head.forward[self.level] is None:
                self.level -= 1

            return deleted_node.value
        else:
            # Key not found
            return None  

    def display(self):
        """ Display the elements in the skip list. """
        for level in range(self.level + 1):
            node = self.head.forward[level]
            elements = []
            while node:
                elements.append((node.key, node.value))
                node = node.forward[level]
            print(f"Level {level}: {elements}")

In [5]:
skip_list = SkipList()
skip_list.insert(10, "A")
skip_list.insert(20, "B")
skip_list.insert(15, "C")
skip_list.insert(5, "D")
skip_list.insert(25, "E")

print("Search 15:", skip_list.search(15))  
print("Delete 20:", skip_list.delete(20))  

skip_list.display()

Search 15: C
Delete 20: B
Level 0: [(5, 'D'), (10, 'A'), (15, 'C'), (25, 'E')]
Level 1: [(5, 'D'), (10, 'A')]
Level 2: [(10, 'A')]
